In [ ]:
# Data availability
##' Time-series transcriptome data of used in this script is available in the NCBI Gene Expression Omnibus 
##' with the accession number of GSE270332.
##' K562 human cell line in RPMI + 10% BCS (bovine calf serum) + P/S
##' source website : https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE270332

In [2]:
# Loading Packages
import numpy as np
import scanpy as sc
import decoupler as dc
import pandas as pd
import subprocess

/Users/ogawaakane/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# import raw data
transcriptome_raw = pd.read_table("../data/GSE270332_k562_timeseries.tsv")
transcriptome_raw

,time_points,0,2,4,6,8,10,12,14,16,...,28,30,32,34,36,38,40,42,44,46
0,ENSG00000100297,89.55440,112.27900,120.07100,116.91100,93.76150,83.71720,73.51850,71.77830,52.977900,...,102.33200,95.33530,82.677700,73.47040,69.41870,66.22850,71.44990,77.17190,72.10020,82.042500
1,ENSG00000111737,33.67870,33.96490,34.59740,43.82600,37.86600,34.52640,33.47520,32.97790,27.893800,...,36.77900,36.76860,37.126200,33.79380,33.67190,33.17470,33.09310,30.38690,29.95750,28.191300
2,ENSG00000148297,10.97540,11.49190,13.18110,15.92540,14.30110,13.62060,10.78080,10.41470,7.406190,...,14.31810,14.38440,11.566700,12.21660,11.10520,10.08040,10.77470,10.92540,10.12760,10.430500
3,ENSG00000105701,93.00610,95.56310,91.36910,100.91600,95.96990,91.28590,90.85310,89.05840,59.848500,...,98.71720,102.14400,98.021700,96.24640,88.68900,86.44870,88.13930,85.72940,90.04630,91.358000
4,ENSG00000197818,3.91415,4.43319,5.50687,4.99821,4.52989,4.36631,3.55905,3.57020,3.304050,...,4.50633,3.93127,4.220080,3.59540,3.69579,3.36569,3.35434,3.49828,3.60724,3.720270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12280,ENSG00000283013,5.83628,6.00153,6.27504,4.87649,4.58706,4.84714,4.31440,5.60865,2.435130,...,5.77533,5.37708,3.937330,4.46126,6.28437,5.13435,4.57600,6.61901,6.16096,6.403760
12281,ENSG00000283023,3.30684,4.95597,4.42905,3.73530,3.86603,3.51884,3.27480,3.34539,4.251510,...,2.67732,2.88747,3.027450,3.97477,3.23287,3.26197,2.69672,2.65821,3.56394,3.071240
12282,ENSG00000283041,3.46199,3.24993,2.81854,2.32530,2.79339,2.55289,3.21661,3.38800,1.162450,...,3.25894,3.25385,2.130910,2.18677,2.79702,2.72033,2.47282,2.48513,2.27583,2.796920
12283,ENSG00000283050,1.74025,2.42216,2.15528,1.70274,1.55680,1.53613,1.81176,1.62393,1.642280,...,1.50250,2.05978,1.885340,1.63446,1.60809,1.48442,1.66574,1.88539,2.24402,1.556810


In [27]:
# データフォーマットを整える
# この処理はデータごとに異なるので各自で修正する

import mygene

df = transcriptome_raw.rename(columns={"time_points": "ensembl_id"})  
# Ensembl ID → Gene Symbol に変換
mg = mygene.MyGeneInfo()
results = mg.querymany(df["ensembl_id"].tolist(),
                       scopes="ensembl.gene",
                       fields="symbol",
                       species="human")
map_df = pd.DataFrame(results)[["query", "symbol"]]
df = df.merge(map_df, left_on="ensembl_id", right_on="query", how="left")
df = df.drop(columns="query").rename(columns={"symbol": "gene_symbol"})
df = df.dropna(subset=["gene_symbol"])
df = df.groupby("gene_symbol").sum(numeric_only=True)
df


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
1 input query terms found dup hits:	[('ENSG00000278932', 3)]
214 input query terms found no hit:	['ENSG00000267896', 'ENSG00000250400', 'ENSG00000130723', 'ENSG00000248103', 'ENSG00000272993', 'ENS


,0,2,4,6,8,10,12,14,16,18,...,28,30,32,34,36,38,40,42,44,46
gene_symbol,,,,,,,,,,,,,,,,,,,,,
7SK,5.98065,6.11505,9.65210,7.981330,10.757000,11.56310,7.31086,10.02990,6.916020,10.325100,...,1.458200,3.91163,5.98145,6.25282,6.40677,5.75252,7.20874,6.07641,6.25360,26.542900
A1BG,2.56841,2.97777,3.11714,1.465890,1.865050,2.61019,1.92735,1.73854,0.539188,0.178983,...,1.843760,2.59564,2.39676,2.81154,3.44170,3.32603,3.32796,3.71619,3.54308,3.531030
A1BG-AS1,1.20302,1.13011,2.05113,1.325590,1.307360,1.22198,1.49377,1.51476,1.031690,1.023800,...,1.749600,1.50393,1.30041,1.74945,1.67969,1.35156,1.31586,1.64721,1.53177,1.514130
AAAS,54.13820,71.55640,67.61290,92.491300,70.797600,72.96250,66.76870,62.19630,70.749200,63.991400,...,50.159300,47.16350,60.47160,54.17890,45.04680,46.15910,45.88310,44.52080,43.31700,51.982300
AACS,3.59843,3.80771,3.42496,3.954570,3.799670,3.77962,3.30167,3.91678,2.409270,3.034240,...,6.044210,5.68689,4.85886,5.54650,4.98244,5.67251,5.93468,5.52658,5.41401,5.034300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,8.31121,7.27806,8.79881,7.014250,8.056830,8.06953,8.32515,8.29926,10.929000,11.421100,...,9.007850,8.11187,8.37865,8.29881,8.86345,9.01288,9.39868,8.89768,9.08808,9.121220
ZYX,41.34110,48.42240,49.70430,81.549800,41.643500,36.67870,31.96900,32.49980,56.208300,52.182100,...,46.613900,43.83300,90.08100,55.50320,43.60300,42.81410,47.29550,47.81420,46.78080,51.649300
ZZEF1,5.97939,6.48600,5.93625,7.293720,6.130440,4.93886,5.01184,4.53384,4.258100,4.712930,...,5.837000,5.16895,5.75013,5.76472,5.94003,4.88588,4.84172,4.73421,4.85647,4.997480


In [ ]:
# calculate z score
expr_z = df.sub(df.mean(axis=1), axis=0).div(df.std(axis=1), axis=0) # (値 - 行平均) / 行標準偏差
expr_z = expr_z.dropna(how="any") # NaNを含む行を削除
expr_z

,0,2,4,6,8,10,12,14,16,18,...,28,30,32,34,36,38,40,42,44,46
gene_symbol,,,,,,,,,,,,,,,,,,,,,
7SK,-0.322570,-0.294559,0.442621,0.094405,0.672901,0.840905,-0.045332,0.521361,-0.127623,0.582886,...,-1.265124,-0.753788,-0.322403,-0.265845,-0.233759,-0.370116,-0.066616,-0.302612,-0.265683,3.962945
A1BG,0.147009,0.590761,0.741840,-1.048139,-0.615444,0.192299,-0.547910,-0.752583,-2.052698,-2.443166,...,-0.638523,0.176526,-0.039063,0.410565,1.093668,0.968280,0.970372,1.391219,1.203565,1.190503
A1BG-AS1,-1.029655,-1.304395,2.166204,-0.567785,-0.636479,-0.958209,0.065953,0.145048,-1.675262,-1.704994,...,1.029975,0.104238,-0.662668,1.029410,0.766539,-0.469924,-0.604450,0.644148,0.209145,0.142674
AAAS,-0.310045,1.121147,0.797124,2.841294,1.058799,1.236681,0.727759,0.352061,1.054822,0.499558,...,-0.636977,-0.883131,0.210348,-0.306701,-1.057053,-0.965659,-0.988337,-1.100273,-1.199185,-0.487188
AACS,-0.937153,-0.737045,-1.103020,-0.596621,-0.744732,-0.763904,-1.220907,-0.632755,-2.074197,-1.476617,...,1.401439,1.059779,0.268038,0.925542,0.386202,1.046029,1.296709,0.906495,0.798858,0.435789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,-0.451919,-1.486788,0.036493,-1.751036,-0.706722,-0.694001,-0.437955,-0.463888,2.170227,2.663146,...,0.245880,-0.651590,-0.384366,-0.464339,0.101240,0.250919,0.637361,0.135527,0.326244,0.359439
ZYX,-0.495431,0.027475,0.122135,2.473709,-0.473101,-0.839718,-1.187497,-1.148301,0.602411,0.305103,...,-0.106071,-0.311421,3.103680,0.550344,-0.328405,-0.386660,-0.055739,-0.017437,-0.093746,0.265760
ZZEF1,0.842171,1.561493,0.780918,2.708353,1.056643,-0.635249,-0.531627,-1.210326,-1.601842,-0.956041,...,0.639995,-0.308551,0.516651,0.537367,0.786285,-0.710474,-0.773176,-0.925826,-0.752233,-0.552016


In [ ]:
# obs
obs = pd.DataFrame(index=expr_z.columns)  
obs.index.name = "sample"  
obs["time"] = obs.index.astype(int)
# expr_z は gene_symbol を行、time を列に持つデータフレーム
X = expr_z.T.values  # AnnData の構造に合わせるために転置
# var
var = pd.DataFrame(index=expr_z.index)  
var.index.name = "gene_symbol"
# AnnData 
adata = sc.AnnData(X=X, obs=obs, var=var)

In [32]:
# Import net
subprocess.run(["Rscript", "get_collectri.R"], check=True)
collectri = pd.read_csv('../data/collectri.csv')
collectri



次のパッケージを付け加えます: ‘dplyr’

以下のオブジェクトは ‘package:stats’ からマスクされています:

    filter, lag

以下のオブジェクトは ‘package:base’ からマスクされています:

    intersect, setdiff, setequal, union

[2025-09-08 12:42:52] [SUCCESS] [OmnipathR] Loaded 64958 interactions from cache.
[2025-09-08 12:42:54] [SUCCESS] [OmnipathR] Downloaded 232 interactions.
警告メッセージ:
OmnipathR::import_tf_mirna_interactions(genesymbols = TRUE, resources = "CollecTRI",  で:
  'OmnipathR::import_tf_mirna_interactions' は廃止予定です
'tf_mirna' を代わりに使って下さい
help("Deprecated") を見て下さい


,source,target,weight
0,MYC,TERT,1
1,SPI1,BGLAP,1
2,SMAD3,JUN,1
3,SMAD4,JUN,1
4,STAT5A,IL2,1
...,...,...,...
43173,NFKB,hsa-miR-143-3p,1
43174,AP1,hsa-miR-206,1
43175,NFKB,hsa-miR-21-5p,1
43176,NFKB,hsa-miR-224-5p,1


In [33]:
# ===== run ULM =====
dc.run_ulm(
    mat=adata,
    net=collectri,
    use_raw=False,     # X を使う
    verbose=True,
    min_n=1
)

Running ulm on mat with 24 samples and 11473 targets for 1013 sources.


In [34]:
# ===== get activity scores =====
score = dc.get_acts(adata, obsm_key='ulm_estimate')
score_df = score.to_df()

In [36]:
# save TF activities
decoupleR_TF = score_df.T
decoupleR_TF.to_csv('../results/tables/decoupleR_TF.csv', index=True)
decoupleR_TF

sample,0,2,4,6,8,10,12,14,16,18,...,28,30,32,34,36,38,40,42,44,46
ABL1,1.747755,-0.125556,-2.003998,-0.389571,0.623947,-0.635417,-0.161983,-0.234867,-1.441498,-1.603185,...,-0.069752,0.884395,-0.087322,1.759140,1.090478,0.529211,0.895447,1.348185,2.994454,3.715083
ADNP,-1.346133,0.286577,-0.511023,0.996136,-1.502125,-2.061647,-2.237794,-0.925382,0.902670,-0.049890,...,0.140963,1.054130,1.463418,0.182388,-0.632241,-0.515918,-0.429535,-0.703141,0.507815,0.736905
ADNP2,-1.251440,-0.431609,0.125248,-1.480702,0.072989,1.216502,-0.900848,2.245575,-0.995812,-1.167102,...,2.035088,0.068051,-1.319071,-0.712826,1.838018,1.454768,2.060070,1.461319,0.595096,0.323726
AEBP1,-1.295464,-0.382205,0.699071,-0.647783,-0.456142,-0.152447,0.109395,0.319658,-0.156313,-0.060833,...,-0.731398,0.606720,-0.453817,0.242474,0.968581,0.473946,1.215865,1.370104,-0.195889,1.304878
AHR,1.615153,0.510234,-0.105645,1.016298,1.250900,1.547011,1.091773,1.964162,0.097759,-0.322876,...,-1.565566,-0.164956,-1.119190,0.064520,-0.596694,-0.879246,-0.406102,0.191165,0.376625,0.841403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF85,-0.456187,0.109575,-0.291635,2.142234,-0.257034,0.115868,0.553258,0.633422,-1.237696,-1.478251,...,0.471939,0.971931,1.634436,1.124814,-0.204019,-0.200630,-0.395432,-0.425393,-0.278837,0.323037
ZNF91,1.012986,1.112900,-0.740339,-0.233776,-0.997871,-0.849602,-0.813738,-0.975889,-0.941718,-0.541876,...,-1.038324,-0.802467,1.034790,0.431356,1.231101,1.681306,0.954911,1.207963,1.302413,0.384596
ZSCAN21,-1.916008,-0.157328,0.863422,2.073975,-0.422311,0.175403,-0.803181,-1.296194,0.360708,-0.296046,...,-0.916393,0.544794,2.005340,0.253868,0.986606,-0.039440,2.368321,0.269371,-0.197001,-0.382050
ZXDA,-0.268960,-0.150477,-0.592915,-1.331434,-1.639216,-1.559954,-1.301481,-0.799653,-0.928155,-0.943727,...,0.538524,0.898016,-0.383448,0.785653,1.534861,2.397137,2.029676,1.871220,1.838906,1.318070


In [38]:
# draw lineplots 
subprocess.run(["Rscript", "TF_lineplot.R"], check=True)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
警告メッセージ:
1: パッケージ ‘ggplot2’ はバージョン 4.4.1 の R の下で造られました 
2: パッケージ ‘purrr’ はバージョン 4.4.1 の R の下で造られました 
3: パッケージ ‘lubridate’ はバージョン 4.4.1 の R の下で造られました 


CompletedProcess(args=['Rscript', 'TF_lineplot.R'], returncode=0)